In [2]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
import logging

import math
import os

import cv2
import numpy as np
import openslide
from scipy import signal
from skimage import morphology
from skimage.filters import threshold_local

import torchvision
from torchvision import transforms

np.set_printoptions(threshold=np.inf)

from skimage.color import rgb2gray, rgb2hed
from skimage.exposure import rescale_intensity
from skimage.util import dtype

from multiprocessing import Pool
import tqdm
from tifffile import memmap

OSError: libopenslide.so.0: cannot open shared object file: No such file or directory

In [ ]:
def rgb2gray(rgb):
    r, g, b = rgb[:, :, 0], rgb[:, :, 1], rgb[:, :, 2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

def Color_Deconvolution(Img, Channel):
    # Channel 0 -> Hema
    # Channel 1 -> Eosin
    # Channel 2 -> DAB
    Img = rgb2hed(Img)
    Img = Img[:, :, Channel]
    Img = rescale_intensity(Img, out_range=(0, 1))
    Img = np.uint8(Img * 255)
    return Img



def Registration_Translation(Target_Img, Moving_Img):
    Target_Img = rgb2gray(Target_Img)
    Moving_Img = rgb2gray(Moving_Img)

    adaptive_thresh = threshold_local(Target_Img, 131, offset=7)
    Target_Img_B = Target_Img < adaptive_thresh

    Target_Img_B = morphology.remove_small_objects(Target_Img_B, 200)
    # Target_Img_B = morphology.remove_small_holes(Target_Img_B, 200)
    # Target_Img_B = morphology.binary_dilation(Target_Img_B, square(3))

    adaptive_thresh = threshold_local(Moving_Img, 131, offset=7)
    Moving_Img_B = Moving_Img < adaptive_thresh
    Moving_Img_B = morphology.remove_small_objects(Moving_Img_B, 200)  # 200
    # Moving_Img_B = morphology.remove_small_holes(Moving_Img_B, 400)

    Cross_Corr = signal.fftconvolve(Target_Img_B, Moving_Img_B[::-1, ::-1])
    if np.max(Cross_Corr) != 0:
        Cross_Corr = Cross_Corr / np.max(Cross_Corr)
    else:
        Cross_Corr = Cross_Corr / 1

    ind = np.unravel_index(np.argmax(Cross_Corr, axis=None), Cross_Corr.shape)
    # Moving_Img_Translation-> + , plus the tranlation to moving image coord , Moving_Img_Translation-> - minus the tranlation to moving image coord
    Moving_Img_Translation = np.array(ind) - np.array(Moving_Img_B.shape)
    return Target_Img_B, Moving_Img_B, Moving_Img_Translation, Cross_Corr


def Local_Registration_Translation(Target_Img,
                                   Moving_Img):  # Calculate the image Translation
    Target_Img = Color_Deconvolution(Target_Img, 1)
    Moving_Img = Color_Deconvolution(Moving_Img, 2)
    #    Target_Img = rgb2gray(Target_Img)
    #    Moving_Img = rgb2gray(Moving_Img)
    adaptive_thresh = threshold_local(Target_Img, 131, offset=7)
    #  adaptive_thresh = threshold_local(Target_Img, 151, offset=10)
    Target_Img_B = Target_Img < adaptive_thresh
    Target_Img_B = morphology.remove_small_objects(Target_Img_B, 200)
    Target_Img_B = Target_Img_B.astype(int)
    h, w = Target_Img_B.shape
    Target_Img_B[Target_Img_B == 0] = -1

    #  adaptive_thresh = threshold_local(Moving_Img, 51, offset=3)
    adaptive_thresh = threshold_local(Moving_Img, 131, offset=7)
    Moving_Img_B = Moving_Img < adaptive_thresh
    Moving_Img_B = morphology.remove_small_objects(Moving_Img_B, 200)  # 200
    Moving_Img_B = Moving_Img_B.astype(int)
    h, w = Moving_Img_B.shape
    Moving_Img_B[Moving_Img_B == 0] = -1

    Cross_Corr = signal.fftconvolve(Target_Img_B, Moving_Img_B[::-1, ::-1])
    if np.max(Cross_Corr) != 0:
        Cross_Corr = Cross_Corr / np.max(Cross_Corr)
    else:
        Cross_Corr = Cross_Corr / 1

    ind = np.unravel_index(np.argmax(Cross_Corr, axis=None), Cross_Corr.shape)
    # Moving_Img_Translation-> + , plus the tranlation to moving image coord , Moving_Img_Translation-> - minus the tranlation to moving image coord
    Moving_Img_Translation = np.array(ind) - (np.array(Moving_Img_B.shape) - 1)
    return Moving_Img_Translation

# Patch Visualization

In [ ]:
#patient_id = '2102(1)'
#dir = 'result_patch_rgb2gray(51_3)_100percent'


################### No Restain #################
patient_id = 'S07-65222'
dir = 'result_patch_LCA'

region_idx = np.array([os.path.splitext(os.path.basename(i))[0][0:6] == 'region' for i in glob.glob('{}/{}/*'.format(dir, patient_id))])
region_path = np.array(glob.glob('{}/{}/*'.format(dir, patient_id)))[region_idx]
region_name = [os.path.basename(i) for i in region_path]

In [1]:
patient_id = [i.split('/')[-1] for i in sorted(glob.glob('result_patch_LCA/*'))]



for patient in patient_id:
    
    patient_id = patient
    dir = 'result_patch_LCA'

    region_idx = np.array([os.path.splitext(os.path.basename(i))[0][0:6] == 'region' for i in glob.glob('{}/{}/*'.format(dir, patient_id))])
    region_path = np.array(sorted(glob.glob('{}/{}/*'.format(dir, patient_id))))[region_idx]
    region_name = sorted([os.path.basename(i) for i in region_path])

    print("--- Patient : {} ---".format(patient))
    print('\n')
    
    for r in range(len(region_path)):

        print('Region : {}'.format(region_name[r]))

        HE_path = sorted(glob.glob(os.path.join(region_path[r], 'HE_{}/*'.format(patient_id))))
        IHC_path = sorted(glob.glob(os.path.join(region_path[r], 'IHC_{}/*'.format(patient_id))))

        HE_binary_path = sorted(glob.glob(os.path.join(region_path[r], 'HE_binary{}/*'.format(patient_id))))
        IHC_binary_path = sorted(glob.glob(os.path.join(region_path[r], 'IHC_binary{}/*'.format(patient_id))))


        fig = plt.figure(figsize=(20, 20)) # rows*cols 행렬의 i번째 subplot 생성
        fig1 = plt.figure(figsize=(20, 20))
        rows = 5
        cols = 5


        for i in range(5):


            try:
                HE_image = cv2.imread(HE_path[i])
                HE_image = cv2.resize(HE_image, (224, 224))
                HE_image = cv2.cvtColor(HE_image, cv2.COLOR_BGR2RGB)

                IHC_image = cv2.imread(IHC_path[i])
                IHC_image = cv2.resize(IHC_image, (224, 224))
                IHC_image = cv2.cvtColor(IHC_image, cv2.COLOR_BGR2RGB)    


                ax = fig.add_subplot(rows, cols, i+1)
                ax.imshow(HE_image)

                ax1 = fig1.add_subplot(rows, cols, i+1)
                ax1.imshow(IHC_image)    


                ax.set_xlabel('Region : {}'.format(region_name[r]))
                ax1.set_xlabel('Region : {}'.format(region_name[r]))
            except:
                pass
            

        plt.show()
        print('\n')
        print('\n')

NameError: name 'glob' is not defined